<a href="https://colab.research.google.com/github/mohmaed7777/Novozymes-Enzyme-Stability-Prediction/blob/main/NovozymeAmino_Acid_Properties.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!mkdir ~/.kaggle

In [3]:
! cp kaggle.json ~/.kaggle/

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle competitions download -c novozymes-enzyme-stability-prediction


 71% 5.00M/7.06M [00:00<00:00, 40.9MB/s]
100% 7.06M/7.06M [00:00<00:00, 53.6MB/s]


In [6]:
! unzip novozymes-enzyme-stability-prediction.zip

Archive:  novozymes-enzyme-stability-prediction.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               
  inflating: train_updates_20220929.csv  
  inflating: wildtype_structure_prediction_af2.pdb  


In [7]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from itertools import product 
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingRegressor



In [18]:
Aminoacids = ['A', 'R', 'N','D', 'C','E','Q','G','H','I',
              'L','K','M','F','P','S','T','W','Y','V']

Blocks =[]

maxSize = 4

for k in range(1, maxSize):
  Blocks.append([''.join(i) for i in product(Aminoacids, repeat= k)])

In [17]:
#data = pd.read_csv('', index_col='seq_id')
#dataMod = pd.read_csv('', index_col='seq_id')

In [19]:
data = pd.read_csv('/content/train.csv',index_col="seq_id")
dataMod = pd.read_csv('/content/train_updates_20220929.csv',index_col="seq_id")

#from https://www.kaggle.com/competitions/novozymes-enzyme-stability-prediction/discussion/356251 Petr1zi0's answer

all_features_nan = dataMod.isnull().all("columns")
data = data.drop(index=dataMod[all_features_nan].index)

swap_ph_tm_indices = dataMod[~all_features_nan].index
data.loc[swap_ph_tm_indices, ["pH", "tm"]] = dataMod.loc[swap_ph_tm_indices, ["pH", "tm"]]

data['seqsize'] = [len(val) for val in data['protein_sequence']]

data = data.fillna(data['pH'].max())